# Assignment 9

In [1]:
import numpy as np
import pandas as pd

### Part A

Consider a two period binomial model. Suppose after each period, the stock will either rise in value by ```r``` percent, or fall in value by ```f``` percent. Currently, the stock price is ```P```, and the risk free rate is ```rf```.

Write the function ```european_put```, which calculates the value of a one year European Put option with a strike price of ```K```. 

In [2]:
def european_put(P, K, r, f, rf):
    p = (np.sqrt(1 + rf) - 1 - f) / (r - f)
    put_expiry = max(0, K - (P * ((1 + f) ** 2)))
    put_expiry2 = max(0, K - (P * ((1 + r) ** 2)))
    put_expiry3 = max(0, K - P)
    P_1 = ((p * put_expiry2) + ((1 - p) * put_expiry3)) / np.sqrt(1 + rf)
    P_2 = ((p * put_expiry3 + (1 - p) * put_expiry)) / np.sqrt(1 + rf) 
    P_0 = ((p * P_1) + ((1 - p) * P_2)) / np.sqrt(1 + rf)
    return P_0
            
# european_put(40,40, 0.25, -0.2, 0.1) should return 2.616...

Write the function ```american_put```, which calculates the value of a one year American Put option with a strike price of ```K```. 

In [3]:
def american_put(P, K, r, f, rf):
    p = (np.sqrt(1 + rf) - 1 - f) / (r - f)
    put_expiry = max(0, K - (P * (1 + f)))
    put_expiry2 = max(0, K - (P * (1 + r)))
    P_0 = ((p * put_expiry2) + ((1 - p) * put_expiry)) / np.sqrt(1 + rf)
    return P_0

# american_put(40,40, 0.25, -0.2, 0.1) should return 3.410...

### Part B

In addition to simple calls and puts, a wide variety of exotic options are traded in over-the-counter markets. In most cases, there are no analytic formulas available to value these options, and so numerical techniques must be used. One example of these techniques are Monte Carlo methods. These are particularly useful in certain contexts, such as when option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. This question will explore some basic examples, and it will require relatively large number of paths. Assume that the stock price will be needed at the end of each of the next 12 months, in addition to S0 today, and start by simulating `N` such stock price paths. In particular:

•	You have been provided the function ```gen_random_vars```, which generates random variables that follow a standard normal distribution (Note that you should never actually call ```gen_random_vars``` in any of your functions, this is simply meant to help give you inputs to help test your code). This outputs a table with `N` rows and 12 columns. Denote the value of the random variable on path i for month j by zi,j where i = 1, . . . ,`N` and j = 1, . . . ,12.

Write the function ```create_paths```, which takes as an argument a DataFrame ```random_vars```, the output of the function ```gen_random_vars```, and ```S0```, the initial stock price.

This function will generate a DataFrame with `N` stock price paths $S_{i,j}$ where $i = 1, . . . ,N$ and $j = 0, . . . ,12$. In each case, $S_{i,0} =$ ```S_0```. For the remaining entries in this table, define $∆t = 1/12$ and set

$$ S_{i,j} = S_{i, j-1} \exp [(r - \sigma^2 / 2) \Delta t + \sigma \sqrt{\Delta t} z_{i,j}]$$

 
for i = 1, . . . ,`N` and j = 1, . . . ,12. (If you are wondering why we are using the risk-free rate r here instead of the expected rate of return on the underlying stock, it is because we are using risk-neutral valuation.) 

Note that throughout the rest of this question, $S_T$ on each simulated path i is defined to be equal to $S_{i,12}$.



In [4]:
def gen_random_vars(SEED, N):
    rng = np.random.default_rng(SEED)

    zArr = np.array([rng.normal(size=N)])
    for i in range(11):
        z = rng.normal(size=N)
        zArr = np.r_[zArr,[z]]
    

    randomVariables = pd.DataFrame(zArr.transpose())
    return randomVariables

r_vars = gen_random_vars(727, 10000)
r_vars

0         1         2         3         4         5         6   \
0     1.744763 -0.514250 -0.710679  1.495061  0.413633 -2.361391  0.122873   
1    -0.962309 -0.576888  0.396051  1.595821  0.008168 -0.348428 -0.340074   
2     0.496945  0.579497 -0.315875  0.330535 -0.045317  0.722330  0.760781   
3    -0.282490 -0.940921  1.175841  0.405341 -0.923402  0.977314 -0.128823   
4    -0.400287 -1.035328 -1.143469 -0.976739  0.649610 -0.414282 -0.257923   
...        ...       ...       ...       ...       ...       ...       ...   
9995  0.812176  0.492083  0.234801 -0.822796 -1.171159 -1.472986  0.598065   
9996 -0.714678 -0.710493 -0.644405  0.711664 -0.901012  0.526393 -2.218868   
9997 -1.684857  0.816217  0.048860  0.366472  0.296956 -0.250303  1.962326   
9998 -0.418933  0.839082 -2.012594  1.065439 -0.527060 -0.770415 -0.886027   
9999 -0.471454  0.268310  0.767433 -1.018053 -1.031914 -0.759433  0.806048   

            7         8         9         10        11  
0    -0.095099 -0.884679 -0.148495 -0.508705  1.562580  
1     0.230534 -0.556868 -1.567693 -0.009760  0.885091  
2    -0.070977  0.250281 -0.329167  0.407775  0.864023  
3    -0.688989 -1.411402  0.010026 -1.573026  0.730829  
4     0.762621 -1.664448 -1.159700  0.268761  1.822126  
...        ...       ...       ...       ...       ...  
9995 -0.239307 -0.822385  0.653382  0.729384  0.928624  
9996  2.422271  0.392027  1.610234  0.387898 -0.058958  
9997  1.416475  0.541789  0.660781  1.132385  0.190961  
9998  0.911825 -0.648514 -2.227339 -0.125608 -1.135578  
9999  0.445378  0.758105  0.378315 -0.898852  0.473172  

[10000 rows x 12 columns]

In [5]:
def create_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    output = []
    t = 1/12
    for col, row in random_vars.iterrows():
        listt = [S0]
        for i in range(0, 12):
            listt += [listt[i] * np.exp((rf - (sigma ** 2) / 2) * t + (sigma * np.sqrt(t) * row[i]))]
        output += [listt]       
    df = pd.DataFrame(output, columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])
    return df

# If we let sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35),
# The last column of the first row of sample_paths should equal 71.56...


•	Write the function ```payoff_table```, which returns another DataFrame with `N` rows and 6 columns. The columns will be used to store discounted payoffs at expiration for various different options, as indicated below:

–	European call option, with payoff $max(S_T − K,0)$.

–	European put option, with payoff $max(K - S_T ,0)$.

–	Average price put option, with payoff $max(K − S^-,0)$, where
$S¯ = S_0 + S_1 + · · · + S_{12}$ .

–	Floating lookback call option, with payoff $S_T - S_{min}$, where $S_{min}$ on each path is the minimum of the stock prices $S_0, S_1, . . . , S_{12}$ on that path.

–	Up-and-out call option with barrier ```B```, with payoff $max(S_T - K,0)$ as long as none of the observed monthly stock prices on the given path are greater than or equal to ```B```. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.

–	Up-and-in call option with barrier ```B``` , with payoff $max(S_T - K,0)$ as long as at least one of the observed monthly stock prices on the given path is greater than or equal to ```B```. If this condition is not satisfied, the option payoff is zero.

Note that “discounted payoffs” implies that the entries in this table should have the payoff functions multiplied by $e^{−rT}$ , where T = 1.

This function will also take in a dataframe ```paths```, which is the output from the previous function ```create_paths```.

In [6]:
def payoff_table(paths, K, T, rf, B):
    euro_calls = []
    euro_puts = []
    avg_puts = []
    floating_calls = []
    up_out = []
    up_in = []
    paths["minimum"] = paths.min(axis=1) 
    for col, row in paths.iterrows():
        ec = max((row[12] - K), 0)
        ep = max(K - row[12], 0)
        avg = 0
        count = 0
        for i in range(0, 13):
            avg += row[i] 
            avg_put = max(K - (avg / 13), 0)
            if row[i] >= B:
                count += 1
        if count > 0:
            out = 0
            inn = max((row[12] - K), 0)
        else:
            out = max((row[12] - K), 0)
            inn = 0
        euro_calls += [ec * np.exp(-rf*T)]
        euro_puts += [ep * np.exp(-rf*T)]
        avg_puts += [avg_put * np.exp(-rf*T)]
        floating_calls += [(row[12] - row["minimum"]) * np.exp(-rf*T)]
        up_out += [out * np.exp(-rf*T)]
        up_in += [inn * np.exp(-rf*T)]
    
    answer = pd.DataFrame(data = {
        "European Call" : euro_calls,
        "European Put" : euro_puts,
        "Average Price Put" : avg_puts,
        "Floating Lookback" : floating_calls,
        "Up-and-out" : up_out,
        "Up-and-in" : up_in
    })
    
    return answer
            
# If we let sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
# The payoff of the first European Call should be 1.500...

Write the function ```statistics```, which takes as input a DataFrame ```payoffs```, formatted as one produced by ```payoff_table```, and displays the following statistics for each of the options above (below N is the number of simulated paths):

•	Mean discounted payoff µ. This is the estimated option premium.

•	Standard error of option premium ϕ. This is the standard deviation of the option payoff divided by √N .

•	Lower bound for 95% confidence interval for option premium, which is µ - 1.96ϕ.

•	Upper bound for 95% confidence interval for option premium, which is µ + 1.96ϕ.

The size of the returned DataFrame should have 6 rows and 4 columns. 

In [7]:
def statistics(payoffs):
    
    # Note that the index rows have been renamed here. Ensure that your rows match the order 
    # that the indices have been renamed as.
    mean = payoffs.mean().tolist()
    sd = (payoffs.std() / np.sqrt(len(payoffs))).tolist()
    lower = []
    upper = []
    for i in range(len(mean)):
        lower.append(mean[i] - (1.96 * sd[i]))
        upper.append(mean[i] + (1.96 * sd[i]))
    output = pd.DataFrame(data = {
        "Mean" : mean,
        "Standard Error" : sd,
        "Lower Bound" : lower,
        "Upper Bound" : upper
    }).rename(index={0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}, errors="raise")
    
    return output

# If we call statistics(sample_payoffs), the mean of the European Puts should be 7.370...

One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, there has been a lot of research into variance reduction methods, in an effort to improve the rate of convergence. 

One of the simplest such approaches is called antithetic variates. This relies on the fact that if Z is a standard normal random variable, then so is -Z. To see how this can be used, generate another `N` stock price paths, but this time using $-z_{i,j}$ instead of $z_{i,j}$ throughout.

In order to accomplish this, write the function ```create_negative_paths```. This function has the same arguments as ```create_paths```, but creates the stock prices in the manner described above.

In [8]:
def create_negative_paths(random_vars, S0, rf, sigma):
    output = []
    t = 1/12
    for col, row in random_vars.iterrows():
        listt = [S0]
        for i in range(0, 12):
            listt += [listt[i] * np.exp((rf - (sigma ** 2) / 2) * t + (sigma * np.sqrt(t) * (-row[i])))]
        output += [listt]
            
    df = pd.DataFrame(output, columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])
    return df


Write the function ```antithetic_statistics```.

In this function, you will use ```payoff_table``` to create two tables, one using paths generated from ```create_paths```, and one generated from ```create_negative_paths```.

Next, create a third table which contains the average payoff between the two tables.

Finally, call ```statistics``` on this third table. Return the table produced by ```statistics```.

If you compare this table with the output of `statistics` (as above, without creating two tables), you should notice that Mean, Lower Bound, and Upper Bound columns should have relatively similar values, while the Standard Error column has decreased significantly.

In [36]:
def antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    #sample_paths = create_paths(r_vars, S0, rf, sigma)
    sample_paths2 = create_negative_paths(r_vars, S0, rf, sigma)
    
    return sample_paths2

    table1 = payoff_table(sample_paths, K, T, rf, B)
    table2 = payoff_table(sample_paths2, K, T, rf, B)
    table3 = (table1 + table2) / 2
    table4 = statistics(table3)
    return sample_paths2
    return table4


In [37]:
def correct_create_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*randomVariables[j][i])
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_payoff_table(paths, K, T, rf, B):
    N = len(paths)
    #discounts a payment by a year per the question specifications
    def discount(pmt):
        return pmt*np.exp(-rf * T)

    #functions that calculate payoff for each option given a price path
    def EuropeanCall(path):
        return discount(max(path[12] - K, 0))

    def EuropeanPut(path):
        return discount(max(K-path[12], 0))
    
    def AveragePricePut(path):
        Sbar = path.mean()
        return discount(max(K - Sbar, 0))
    
    def FloatingLookback(path):
        Smin = path.min()
        return discount(path[12] - Smin)
    
    def upAndOut(path, B):
        for i in range(13):
            if(path[i] >= B):
                return 0
        return discount(max(path[12] - K, 0))
    
    def upAndIn(path, B):
        atLeastOne = False
        for i in range(13):
            if(path[i] >= B):
                atLeastOne = True
                break
        if atLeastOne:
            St = path[12]
            return discount(max(St - K, 0))
        else:
            return 0
    
    payoffsArr = np.zeros((N, 6))
    
    for i in range(N):
        row = paths.iloc[i]
        payoffsArr[i][0] = EuropeanCall(row)
        payoffsArr[i][1] = EuropeanPut(row)
        payoffsArr[i][2] = AveragePricePut(row)
        payoffsArr[i][3] = FloatingLookback(row)
        payoffsArr[i][4] = upAndOut(row, 90)
        payoffsArr[i][5] = upAndIn(row, 90)
        
    payoffs = pd.DataFrame(payoffsArr)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    payoffs = payoffs.rename(columns=renameDict, errors="raise")
    return payoffs

def correct_statistics(payoffs):
    N = len(payoffs)
    #Statistics
    statLst = []
    for column in payoffs:
        row = []
        info = payoffs[column]
        mean = info.mean()
        sd = info.std()
        se = sd/(N**0.5)
        row.append(mean)
        row.append(se)
        row.append(mean-1.96*se)
        row.append(mean+1.96*se)
        statLst.append(row)

    stats = pd.DataFrame(statLst)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    stats = stats.rename(index=renameDict, errors="raise")
    renameDict2 = {0: "Mean", 1: "Standard Error", 2: "Lower Bound", 3: "Upper Bound"}
    stats = stats.rename(columns=renameDict2, errors="raise")
    return stats



def correct_create_negative_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*(-randomVariables[j][i]))
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    paths1 = correct_create_paths(random_vars, S0, rf, sigma)
    paths2 = correct_create_negative_paths(random_vars, S0, rf, sigma)
    
    return paths2
    payoffs1 = correct_payoff_table(paths1, K, T, rf, B)
    payoffs2 = correct_payoff_table(paths2, K, T, rf, B)
    
    avg_payoffs = (payoffs1 + payoffs2)/2
    
    return paths2
    return correct_statistics(avg_payoffs)

test_r_vars = gen_random_vars(700, 20)

test_paths = correct_create_paths(test_r_vars, 72.25, 0.04 , 0.35)

test_payoff_table = correct_payoff_table(test_paths, 70, 1, 0.04, 90)

In [38]:
antithetic_statistics(test_r_vars, 72.25, 0.04, 0.35,  70, 1, 90)

0          1          2          3           4          5  \
0     72.25  60.465753  63.577791  68.189988   58.526026  56.031205   
1     72.25  79.486690  84.108312  80.666154   68.532797  68.355108   
2     72.25  68.590360  64.575235  66.551425   64.251695  64.432357   
3     72.25  74.210318  81.467120  72.213429   69.193092  75.824942   
4     72.25  75.098829  83.232666  93.260903  102.751789  96.054071   
...     ...        ...        ...        ...         ...        ...   
9995  72.25  66.440199  63.105833  61.517331   66.731719  74.981325   
9996  72.25  77.522632  83.144884  88.581420   82.289821  89.973169   
9997  72.25  85.506572  78.598673  78.073237   75.102175  72.753380   
9998  72.25  75.240445  69.002336  84.412421   75.663486  79.660752   
9999  72.25  75.640770  73.487516  67.884389   75.105632  83.211491   

              6           7          8           9          10          11  \
0     71.003142   70.003039  70.553850   77.014221   78.040090   82.010686   
1     70.679061   73.020365  71.212991   75.201283   87.952012   87.883026   
2     59.791504   55.269779  55.569065   54.085619   55.815707   53.468059   
3     68.573922   69.349387  74.217457   85.441449   85.203922   99.704355   
4     99.982779  102.440970  94.676278  111.816859  125.494670  121.916682   
...         ...         ...        ...         ...         ...         ...   
9995  86.859623   81.621784  83.471396   90.542914   84.608761   78.458725   
9996  85.162046  106.375067  83.134775   79.764879   67.668469   64.952595   
9997  74.484737   60.980692  52.755658   49.857047   46.554618   41.448154   
9998  85.956909   93.840463  85.430044   91.053788  113.831767  115.081636   
9999  89.688714   82.527704  78.756737   72.820452   69.965513   76.481459   

              12  
0      69.909489  
1      80.222939  
2      48.911668  
3      92.443557  
4     101.237194  
...          ...  
9995   71.305760  
9996   65.225063  
9997   40.584188  
9998  128.844406  
9999   72.782091  

[10000 rows x 13 columns]

In [39]:
correct_antithetic_statistics(test_r_vars, 72.25, 0.04, 0.35,  70, 1, 90)

0          1          2           3          4           5   \
0   72.25  72.233510  73.213880   69.402811  71.422785   82.043820   
1   72.25  72.629720  74.751074   72.967767  74.586231   78.840366   
2   72.25  67.573750  72.132818   72.694835  63.973046   56.559718   
3   72.25  64.590107  63.377624   64.319609  58.902533   69.962695   
4   72.25  69.656742  68.846616   60.000639  81.526298   77.669191   
5   72.25  66.644341  77.137209   68.774118  74.056145   71.939101   
6   72.25  72.905320  78.837435   62.454986  61.182762   56.334430   
7   72.25  64.344882  62.520197   73.255981  74.503809   64.751031   
8   72.25  74.003865  68.373603   64.677182  51.254107   59.229751   
9   72.25  78.979063  76.601629   70.665371  66.586437   82.013815   
10  72.25  66.089598  68.904125   71.127855  61.198590   72.395986   
11  72.25  77.831262  72.724430   62.885190  62.441513   62.503224   
12  72.25  78.888555  90.261552  107.809588  91.221780  110.243906   
13  72.25  76.112631  73.147382   67.486052  70.085630   65.958444   
14  72.25  66.213745  61.870682   60.124358  56.357238   60.207926   
15  72.25  66.402764  68.715480   57.303629  60.516168   58.423760   
16  72.25  78.323191  87.741716   93.645983  96.232106   98.292794   
17  72.25  93.479385  85.791541   96.057555  88.605783   86.394591   
18  72.25  83.331599  68.470183   67.448436  81.164000   75.707580   
19  72.25  76.151393  76.628412   88.646269  92.328405  102.173770   

            6           7           8           9           10          11  \
0    73.884946   70.288677   64.870068   61.840061   60.094251   50.852865   
1    79.543695   88.036341   87.883055   79.777586   75.521535   84.139457   
2    65.048144   65.033235   65.747478   69.461061   67.338399   85.522158   
3    71.300046   77.788218   72.970567   71.985446   63.318265   57.708814   
4    87.617095   79.927873   85.877110   86.762907   93.656207   98.634853   
5    80.395872   73.673747   69.080938   72.297152   74.923928   63.713175   
6    58.640384   63.838988   65.739310   57.234967   53.545181   57.034064   
7    56.066743   49.918229   50.167290   55.273646   48.883906   42.299164   
8    62.672531   67.572820   73.825846   66.356294   70.990513   60.765857   
9    94.003252   95.318875   82.144876   68.268943   79.285432   94.061796   
10   66.994171   64.499817   60.071614   50.069989   48.591325   51.193712   
11   56.030807   48.355463   43.300293   43.494640   39.368962   39.662598   
12  124.166304  126.578987  124.158087  132.609439  129.789762  126.775705   
13   64.343409   89.962506   99.608983  128.599609  142.829023  170.889601   
14   62.339719   49.067909   50.063819   50.143197   59.637021   74.722004   
15   63.073275   49.221271   52.388305   46.854019   55.356363   69.580088   
16  111.664886  125.744998  122.299826  132.201888  134.298229  147.500694   
17   72.423218   71.275595   72.333634   71.449143   69.843806   71.871420   
18   69.929324   63.402013   67.281830   85.098218   87.474909   80.959856   
19  102.792112  107.002673  118.308873  121.977451  136.604302  153.964806   

            12  
0    52.800415  
1    83.163828  
2    91.139584  
3    59.804384  
4    95.099350  
5    64.419771  
6    54.910007  
7    39.177962  
8    58.740302  
9   100.141559  
10   52.054251  
11   35.382879  
12  124.292054  
13  148.147814  
14   74.069598  
15   65.454488  
16  132.487986  
17   79.991649  
18   76.135030  
19  146.702997

In [34]:
create_negative_paths(r_vars, 72.25, 0.04 , 0.35)

0          1          2          3           4          5  \
0     72.25  60.465753  63.577791  68.189988   58.526026  56.031205   
1     72.25  79.486690  84.108312  80.666154   68.532797  68.355108   
2     72.25  68.590360  64.575235  66.551425   64.251695  64.432357   
3     72.25  74.210318  81.467120  72.213429   69.193092  75.824942   
4     72.25  75.098829  83.232666  93.260903  102.751789  96.054071   
...     ...        ...        ...        ...         ...        ...   
9995  72.25  66.440199  63.105833  61.517331   66.731719  74.981325   
9996  72.25  77.522632  83.144884  88.581420   82.289821  89.973169   
9997  72.25  85.506572  78.598673  78.073237   75.102175  72.753380   
9998  72.25  75.240445  69.002336  84.412421   75.663486  79.660752   
9999  72.25  75.640770  73.487516  67.884389   75.105632  83.211491   

              6           7          8           9          10          11  \
0     71.003142   70.003039  70.553850   77.014221   78.040090   82.010686   
1     70.679061   73.020365  71.212991   75.201283   87.952012   87.883026   
2     59.791504   55.269779  55.569065   54.085619   55.815707   53.468059   
3     68.573922   69.349387  74.217457   85.441449   85.203922   99.704355   
4     99.982779  102.440970  94.676278  111.816859  125.494670  121.916682   
...         ...         ...        ...         ...         ...         ...   
9995  86.859623   81.621784  83.471396   90.542914   84.608761   78.458725   
9996  85.162046  106.375067  83.134775   79.764879   67.668469   64.952595   
9997  74.484737   60.980692  52.755658   49.857047   46.554618   41.448154   
9998  85.956909   93.840463  85.430044   91.053788  113.831767  115.081636   
9999  89.688714   82.527704  78.756737   72.820452   69.965513   76.481459   

              12  
0      69.909489  
1      80.222939  
2      48.911668  
3      92.443557  
4     101.237194  
...          ...  
9995   71.305760  
9996   65.225063  
9997   40.584188  
9998  128.844406  
9999   72.782091  

[10000 rows x 13 columns]

In [35]:
correct_create_negative_paths(r_vars, 72.25, 0.04 , 0.35)

0          1          2          3           4          5   \
0     72.25  60.465753  63.577791  68.189988   58.526026  56.031205   
1     72.25  79.486690  84.108312  80.666154   68.532797  68.355108   
2     72.25  68.590360  64.575235  66.551425   64.251695  64.432357   
3     72.25  74.210318  81.467120  72.213429   69.193092  75.824942   
4     72.25  75.098829  83.232666  93.260903  102.751789  96.054071   
...     ...        ...        ...        ...         ...        ...   
9995  72.25  66.440199  63.105833  61.517331   66.731719  74.981325   
9996  72.25  77.522632  83.144884  88.581420   82.289821  89.973169   
9997  72.25  85.506572  78.598673  78.073237   75.102175  72.753380   
9998  72.25  75.240445  69.002336  84.412421   75.663486  79.660752   
9999  72.25  75.640770  73.487516  67.884389   75.105632  83.211491   

             6           7          8           9           10          11  \
0     71.003142   70.003039  70.553850   77.014221   78.040090   82.010686   
1     70.679061   73.020365  71.212991   75.201283   87.952012   87.883026   
2     59.791504   55.269779  55.569065   54.085619   55.815707   53.468059   
3     68.573922   69.349387  74.217457   85.441449   85.203922   99.704355   
4     99.982779  102.440970  94.676278  111.816859  125.494670  121.916682   
...         ...         ...        ...         ...         ...         ...   
9995  86.859623   81.621784  83.471396   90.542914   84.608761   78.458725   
9996  85.162046  106.375067  83.134775   79.764879   67.668469   64.952595   
9997  74.484737   60.980692  52.755658   49.857047   46.554618   41.448154   
9998  85.956909   93.840463  85.430044   91.053788  113.831767  115.081636   
9999  89.688714   82.527704  78.756737   72.820452   69.965513   76.481459   

              12  
0      69.909489  
1      80.222939  
2      48.911668  
3      92.443557  
4     101.237194  
...          ...  
9995   71.305760  
9996   65.225063  
9997   40.584188  
9998  128.844406  
9999   72.782091  

[10000 rows x 13 columns]